### Smoothing Exercise
create a universe of stocks of QTradableStocksUS from the following website

https://www.quantopian.com/posts/working-on-our-best-universe-yet-qtradablestocksus

In [1]:
# Imports
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.data import morningstar as mstar
from quantopian.pipeline.factors import AverageDollarVolume
from quantopian.pipeline.factors.morningstar import MarketCap
from quantopian.pipeline.classifiers.morningstar import Sector
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline import Pipeline
from quantopian.research import run_pipeline

from quantopian.pipeline.factors import SimpleMovingAverage
from quantopian.pipeline.factors import Returns
from zipline.pipeline.factors import DailyReturns

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from pandas import DataFrame as df
import time

from quantopian.pipeline.experimental import QTradableStocksUS

### Pipeline construction using QTradableStocks as Screen

In [2]:


def make_pipeline():
    
    average_day_dv_200 = AverageDollarVolume(window_length = 200)
    market_cap = Fundamentals.market_cap.latest
    price = USEquityPricing.close.latest
    volume = USEquityPricing.volume.latest
    sector = Sector()
    
    # Ranked REturn
    returns = Returns(window_length = 252)
    ranked_return = returns.rank()
    
    # New factors
    # create a factor of 1 year returns, demeaned by sector, rank, zscore
    factor = (
        Returns(window_length = 252).\
        demean(groupby = Sector()).\
        rank().\
        zscore()
    
    )
    
    # Use the newly created factor as an input into SimpleMovingAverage, with a window length of 5
    # Also rank and zscore (don't need to demean by sector)
    factor_smoothed = (
                    SimpleMovingAverage(inputs = [factor], window_length=5).\
                    rank().\
                    zscore()
    )
    
    
    return Pipeline(
        columns={
            'AverageDollarVolume200': average_day_dv_200,
            'MarketCap': market_cap,
            'Price': price,
            'Volume': volume,
            'Sector': sector,
            '1yrReturns': factor,
            '5dAvgReturns': factor_smoothed,
            'ranked_return': ranked_return
        },
        screen = QTradableStocksUS()
    )

In [3]:
# Pipeline is run over this time range and outputs a dataframe indexed by asset name:
START_DATE = '2017'
END_DATE = '2019-07-30'

QTU_pipline2 = run_pipeline(make_pipeline(), START_DATE, END_DATE, chunksize = 252)

In [4]:
QTU_pipline2

1yrReturns  5dAvgReturns  \
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])       -1.663162     -1.694499   
                          Equity(24 [AAPL])       0.242636      0.270176   
                          Equity(31 [ABAX])       0.731982      0.638386   
                          Equity(41 [ARCB])       0.571808      0.623764   
                          Equity(52 [ABM])        1.027217      0.888291   
                          Equity(53 [ABMD])       1.407291      1.366831   
                          Equity(62 [ABT])        0.527013      0.414403   
                          Equity(64 [GOLD])       1.425616      1.415350   
                          Equity(67 [ADSK])       0.628819      0.669624   
                          Equity(76 [TAP])       -0.091286     -0.100028   
                          Equity(84 [ACET])       0.448284      0.184437   
                          Equity(110 [RAMP])      0.945773      0.826479   
                          Equity(114 [ADBE])      0.229062      0.211688   
                          Equity(122 [ADI])       1.038755      1.035176   
                          Equity(128 [ADM])       0.805281      0.791918   
                          Equity(154 [AEM])       0.225669      0.114650   
                          Equity(161 [AEP])      -0.201914     -0.248908   
                          Equity(166 [AES])       0.459821      0.496153   
                          Equity(168 [AET])       1.160243      1.221940   
                          Equity(185 [AFL])       0.063459      0.035558   
                          Equity(197 [AGCO])      0.372269      0.357908   
                          Equity(216 [HES])      -1.352995     -0.460928   
                          Equity(239 [AIG])      -0.360052     -0.421714   
                          Equity(247 [AIN])       0.924055      0.876327   
                          Equity(253 [AIR])       0.368875      0.425702   
                          Equity(266 [AJG])       0.560948      0.520080   
                          Equity(270 [AKRX])     -0.863650     -0.944121   
                          Equity(289 [MATX])     -1.105947     -1.241879   
                          Equity(300 [ALK])      -0.234492     -0.372531   
                          Equity(301 [ALKS])     -0.180196     -0.319359   
...                                                    ...           ...   
2019-07-30 00:00:00+00:00 Equity(52209 [ALLK])   -0.188155     -0.092094   
                          Equity(52211 [TLRY])    1.615452      1.608533   
                          Equity(52233 [BE])     -1.611619     -1.602310   
                          Equity(52241 [TENB])   -0.292296     -0.189788   
                          Equity(52242 [FOCS])   -0.959943     -0.915961   
                          Equity(52244 [BRY])     0.246934      0.548830   
                          Equity(52288 [CWK])          NaN           NaN   
                          Equity(52291 [SONO])         NaN           NaN   
                          Equity(52361 [BPR])          NaN           NaN   
                          Equity(52407 [FTDR])         NaN           NaN   
                          Equity(52408 [GTX])          NaN           NaN   
                          Equity(52424 [ELAN])         NaN           NaN   
                          Equity(52425 [FTCH])         NaN           NaN   
                          Equity(52427 [EB])           NaN           NaN   
                          Equity(52446 [UPWK])         NaN           NaN   
                          Equity(52450 [SVMK])         NaN           NaN   
                          Equity(52491 [GH])           NaN           NaN   
                          Equity(52494 [ESTC])         NaN           NaN   
                          Equity(52508 [ALLO])         NaN           NaN   
                          Equity(52511 [LTHM])         NaN           NaN   
                          Equity(52517 [PLAN])         NaN           NaN   
                    

In [5]:
###  Lets see what stocks had the most gains ytd through
leading_stocks = QTU_pipline2.sort_values(by=['ranked_return'], ascending = False)
leading_stocks.filter(like='2019-07-30', axis = 0).head(50)

1yrReturns  5dAvgReturns  \
2019-07-30 00:00:00+00:00 Equity(49607 [AXSM])    1.729176      1.729251   
                          Equity(32726 [EHTH])    1.723426      1.721784   
                          Equity(42749 [ENPH])    1.717676      1.715561   
                          Equity(50477 [IIPR])    1.715759      1.716806   
                          Equity(50411 [RARX])    1.712564      1.717428   
                          Equity(48628 [NVTA])    1.711926      1.713072   
                          Equity(50288 [TTD])     1.707453      1.706849   
                          Equity(50735 [AYX])     1.703620      1.706227   
                          Equity(44991 [NSTG])    1.708731      1.709338   
                          Equity(50449 [INSG])    1.701703      1.700627   
                          Equity(45758 [VCYT])    1.706814      1.708094   
                          Equity(47321 [CDNA])    1.704898      1.705605   
                          Equity(45429 [FATE])    1.697231      1.691293   
                          Equity(45852 [TNDM])    1.696592      1.694404   
                          Equity(51314 [MDB])     1.685092      1.698138   
                          Equity(31341 [ZIOP])    1.695953      1.686937   
                          Equity(50758 [OKTA])    1.683175      1.689426   
                          Equity(51961 [SMAR])    1.681897      1.683204   
                          Equity(49460 [NVCR])    1.691481      1.672625   
                          Equity(50077 [TWLO])    1.680620      1.682581   
                          Equity(51832 [ZS])      1.677425      1.668270   
                          Equity(51806 [CRON])    1.686370      1.695649   
                          Equity(48254 [WK])      1.674231      1.676981   
                          Equity(47779 [CYBR])    1.671675      1.668892   
                          Equity(51231 [ROKU])    1.679981      1.678848   
                          Equity(50350 [COUP])    1.669119      1.672003   
                          Equity(49275 [RPD])     1.666564      1.660180   
                          Equity(14774 [OLED])    1.664647      1.669514   
                          Equity(49608 [MTCH])    1.664008      1.665158   
                          Equity(4549 [LSCC])     1.659536      1.685070   
                          Equity(46744 [PAYC])    1.658258      1.663291   
                          Equity(51124 [KL])      1.676786      1.673870   
                          Equity(51428 [BAND])    1.655064      1.665781   
                          Equity(50264 [EVBG])    1.654425      1.649602   
                          Equity(1753 [COKE])     1.644841      1.650846   
                          Equity(13098 [SBGI])    1.678703      1.679470   
                          Equity(45667 [VEEV])    1.648675      1.657691   
                          Equity(49126 [WING])    1.673592      1.671381   
                          Equity(46310 [QURE])    1.667203      1.674492   
                          Equity(22364 [EXAS])    1.662730      1.657069   
                          Equity(49060 [SHOP])    1.643564      1.641513   
                          Equity(6449 [RGEN])     1.661453      1.655202   
                          Equity(47330 [GLOB])    1.641008      1.631556   
                          Equity(15752 [ARQL])    1.658897      1.663914   
                          Equity(40848 [FLDM])    1.655703      1.646491   
                          Equity(52100 [AVLR])    1.633980      1.642757   
                          Equity(49178 [GKOS])    1.651869      1.653958   
                          Equity(6683 [SBUX])     1.656341      1.656447   
                          Equity(13639 [ERIE])    1.641647      1.670136   
                          Equity(16437 [VCEL])    1.649953      1.647113   

                                                AverageDollarVolume200  \
2019-07-30 00:00:00+00:00 Equity(49607 [AXSM])            1.519103e+07   
                       

### Evaluate Factors
We'll go over some tools that we can use to evaluate alpha factos. To do so, we'll use the [alphalens library](https://github.com/quantopian/alphalens)

### Import alphelens

In [6]:
import alphalens as al

### Get price data
Note we already got the price data and converted it to returns, which we used to calcualte a factor. We'll retrieve the price dta again, but won't covert these to returns. This is because we'll use alphalens functions that take their input as prices and not returns.

   ### Define the list of assets 
   Just to make sure we get the prices for the stocks that have factor values, we'll get the list of assets, which may be a subset of the original universe

In [7]:
# get list of of stocks in our portfolio (tickers that identify each stock)
assets = QTU_pipline2.index.levels[1].values.tolist()
len(assets)

2714

In [8]:
stock_prices = QTU_pipline2.Price.unstack().fillna(0)
stock_prices

,Equity(2 [ARNC]),Equity(24 [AAPL]),Equity(31 [ABAX]),Equity(39 [DDC]),Equity(41 [ARCB]),Equity(52 [ABM]),Equity(53 [ABMD]),Equity(62 [ABT]),Equity(64 [GOLD]),Equity(67 [ADSK]),...,Equity(52517 [PLAN]),Equity(52525 [REZI]),Equity(52529 [ACA]),Equity(52537 [STNE]),Equity(52548 [TWST]),Equity(52553 [SWI]),Equity(52571 [YETI]),Equity(52592 [LIN]),Equity(52594 [ETRN]),Equity(52603 [APHA])
2017-01-03 00:00:00+00:00,18.550,115.840,52.74,0.0,27.750,40.680,112.70,38.420,15.990,74.01,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-04 00:00:00+00:00,19.190,116.140,53.57,0.0,28.850,40.820,112.37,39.030,16.400,76.17,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-05 00:00:00+00:00,19.690,116.020,53.40,0.0,29.100,41.450,115.74,39.300,16.395,77.51,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-06 00:00:00+00:00,20.090,116.610,52.61,0.0,28.300,40.580,114.71,39.700,17.380,76.93,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-09 00:00:00+00:00,20.700,117.910,53.59,0.0,28.400,39.790,115.33,40.780,16.920,79.30,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-10 00:00:00+00:00,20.400,119.000,55.05,0.0,28.100,39.410,117.11,40.750,16.970,79.59,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-11 00:00:00+00:00,20.710,119.110,48.59,0.0,29.300,39.570,112.24,41.015,16.795,79.98,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-12 00:00:00+00:00,21.090,119.740,50.18,0.0,29.850,40.220,111.27,41.080,16.680,80.87,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-13 00:00:00+00:00,21.000,119.250,49.47,0.0,29.850,39.770,112.41,40.710,16.890,80.63,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
2017-01-17 00:00:00+00:00,21.120,119.040,49.16,0.0,31.350,40.480,115.65,40.930,16.885,81.32,...,0.00,0.000,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00


In [9]:
stock_prices.shape

(647, 2714)

### compare to the factor data for a single stock

In [10]:
stock_index_name = QTU_pipline2.index.get_level_values(1)[1]
single_stock_factor_df = QTU_pipline2[np.in1d(QTU_pipline2.index.get_level_values(1), stock_index_name)]

In [11]:
single_stock_factor_df.head(2)

,,1yrReturns,5dAvgReturns,AverageDollarVolume200,MarketCap,Price,Sector,Volume,ranked_return
2017-01-03 00:00:00+00:00,Equity(24 [AAPL]),0.242636,0.270176,3.048351e+09,6.175885e+11,115.84,311,23251752.0,3760.0
2017-01-04 00:00:00+00:00,Equity(24 [AAPL]),0.332272,0.290450,3.044491e+09,6.193482e+11,116.14,311,23411860.0,4009.0


### Quiz 1:
if you have factor values calcualted before time t, what price would you use to calcuate the factor return on that data?

### Answer 1:
Use data from time t to time t+1 to calcuate forward returns

### Prepare data for use in alphalens
Alphalens make sure the data is formatted properly so that other neat alphalens functions can work with the data. For instance, it lines up th price data and facto rdata and calculates forward returns taht are associated with each factor value. We'll use [alphalens.utils.get_clean_factor_and forward_returns](https://github.com/quantopian/alphalens/blob/master/alphalens/utils.py)

The source code describes what it's used for (im showing just the parameters that we'll use here):
```
def get_clean_factor_and_forward_returns(factor,
                                         prices,
                                         ...
                                         periods=(1, 5, 10),
                                         ...
                                         ):

...
```
We'll give it three inputs: the factor, prices, and periods.
* The factor is the Series containing the factor scores for each stock on each date.
* The prices are the Series of prices for each stock on eahc date (the same dates as for the factor). Note that if the period we give is greater than 1, we'll want ot make sure to pad our price data by the period amount so that forward returns can be calcualted. Keep reading for details about "periods" parameter
* periods: this is the period for which we'll compute forward returns. For instance if prices and facto rdata have one data point per day (daily data), and if we wich to calculate the return of our factor-weighted portfolio every day, then the period woudl be 1, and hte inptu as a list [1]. if we want ot calculate the weekly return we would input[5]. If we wanted both daily and weekly, we coudl input [1,5]

* returns: multi-index Pandas DataFrame containing the cleaned version of the data.

###  Quiz 2:
What alphalens funciton does get_clean_factor_and_forward_returns call to get the forward returns?

### Answer 2:
The function `compute_forward_returns` computes forward returns.

### Quiz 3 
Clean and line up the factors and forward returns using alphlens

In this case we hve the unsmoothed and smoothed factors

In [15]:
df = QTU_pipline2[['1yrReturns', '5dAvgReturns']]
df.head()

1yrReturns  5dAvgReturns
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])    -1.663162     -1.694499
                          Equity(24 [AAPL])    0.242636      0.270176
                          Equity(31 [ABAX])    0.731982      0.638386
                          Equity(41 [ARCB])    0.571808      0.623764
                          Equity(52 [ABM])     1.027217      0.888291

In [20]:
factor_names = df.columns

# usa a dictionary to store each dataframe, one for each factor and its associated forward returns
factor_data = {}

for factor_name in factor_names:
    print("Formating factor data for: " + factor_name)
    
    # Todo: get clean factor and forward returns for each factor
    # choose a single period returns (daily returns)
    
    factor_data[factor_name] = al.utils.get_clean_factor_and_forward_returns(
        factor = df[factor_name],
        prices = stock_prices,
        periods = [1]
    )

Formating factor data for: 1yrReturns
Dropped 1.3% entries from factor data: 1.3% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!
Formating factor data for: 5dAvgReturns
Dropped 1.3% entries from factor data: 1.3% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


### Inspect the cleaned data

In [22]:
cleaned_smoothed_factor = factor_data[factor_names[1]]
cleaned_smoothed_factor.head()

1D    factor  \
date                      asset                                   
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])   0.034501 -1.694499   
                          Equity(24 [AAPL])  0.002590  0.270176   
                          Equity(31 [ABAX])  0.015738  0.638386   
                          Equity(41 [ARCB])  0.039640  0.623764   
                          Equity(52 [ABM])   0.003441  0.888291   

                                             factor_quantile  
date                      asset                               
2017-01-03 00:00:00+00:00 Equity(2 [ARNC])                 1  
                          Equity(24 [AAPL])                3  
                          Equity(31 [ABAX])                4  
                          Equity(41 [ARCB])                4  
                          Equity(52 [ABM])                 4

### Quiz 4
What do you think the '1D' column represents?

### Answer 4:
The 1D column represents the forward returns